In [26]:
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import check_random_state
import sklearn
from sklearn.metrics import roc_auc_score
import random
random.seed(0)

In [46]:
def getMNIST():
    t0 = time.time()
    train_samples = 60000
    # Load data from https://www.openml.org/d/554
    X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
    X = X.reshape((X.shape[0], -1))
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size=10000)
    scaler = StandardScaler()
    #scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    print("==================")
    X_train=list(X_train)
    X_test=list(X_test)
    print(len(X_train))
    print(len(X_test))
    print("==================")
    
    return X_train, y_train, X_test, y_test

In [47]:
def make_38_Subset(X_train, y_train, X_test, y_test):
    templist = []
    for tup in zip(X_train, y_train):
        if(tup[1]=='8' or tup[1]=='3'):
            templist.append(list(tup))

    testList = []
    for tup in zip(X_test, y_test):
        if(tup[1]=='8' or tup[1]=='3'):
            testList.append(list(tup))

    print("3/8 Training set size = "+str(len(templist)))
    print("3/8 Test set size = "+str(len(testList)))
    
    X_test=[]
    y_test=[]

    for tup in testList:
        X_test.append(tup[0])
        y_test.append(tup[1])


    seed_size=int(0.1*len(templist))
    seed_list=templist[0:seed_size]
    print("Initial Seed set size = "+str(len(seed_list)))
 
    unlabelled_list=templist[seed_size:]
    print("Unlabelled dataset size = "+str(len(unlabelled_list)))
    
    return seed_list, unlabelled_list, testList


In [48]:
def random_instance_generate(seed_set, unlabelled_list, batch_size):
    random_elements=[]
    print("Before Sampling Unlabelled Data Size: "+str(len(unlabelled_list)))
    print("Before Sampling Seed Data Size: "+str(len(seed_set)))
    random.shuffle(unlabelled_list)
    for i in range(batch_size):
        rand_elem = random.choice(unlabelled_list)
        random_elements.append(rand_elem)
    for elem in random_elements:
        seed_set.append(elem)
        ul=[]
    for i in random_elements:
        ul.append(list(i[0]))
        
    new_unlabelled=[]
    for elem in unlabelled_list:
        if(list(elem[0]) in ul):
            continue
        new_unlabelled.append(elem)
    print("After Sampling Unlabelled Data Size : "+str(len(new_unlabelled)))
    print("After Sampling Seed Data Size : "+str(len(seed_set)))
    return new_unlabelled, seed_set

In [49]:
import numpy as np
import numpy.random as npr

def select_random_from_unlabeled(u, batch_size):
    selected = set(npr.choice(len(u), batch_size, replace=False)) #these will no longer be unlabeled
    modified_u = [] # modified unlabeled
    del_s = [] # new points to add to s
    
    i=0
    for x in u:
        if i in selected:
            del_s.append(x)
        else:
            modified_u.append(x)
            
        i=i+1
    return del_s, modified_u

In [50]:
def train_model(data, X_test, y_test):
    print("--------------------Start------------------")
    clf = LogisticRegression(C=50. / train_samples, penalty='l2', solver='saga', tol=0.1)
    print("Training with seed size: " + str(len(data)))
    #random.shuffle(data)
    X_train=[]
    y_train=[]
    for i in range(len(data)):
        try:
            #print(data[i][1])
            X_train.append(list(data[i][0]))
            y_train.append(data[i][1])
        except:
            pass
        
    clf.fit(X_train, y_train)
    print("Accuracy on the Test Set is : "+str(sklearn.metrics.accuracy_score(y_test, clf.predict(X_test))))
    print("--------------------End------------------")
    return(clf)


In [51]:
print ("Making MNIST data...")
X_train, y_train, X_test, y_test = getMNIST()

Making MNIST data...
60000
10000


In [52]:
print ("Getting MNIST 3/8 classification data...")
s, u, testList = make_38_Subset(X_train, y_train, X_test, y_test)

batch_size = 1000
#stopping_criteria = int(len(unlabelled_list)/100)
#print(len(X_test))

niters = 10

for i in range(niters):
    
    print("|S|_{} = {}, |U|_{} = {}".format(i, len(s), i, len(u)))
    train_model(s, X_test, y_test)
    
    #unlabelled_list, seed_list = random_instance_generate(seed_list, unlabelled_list, batch_size)    
    
    del_s, u = select_random_from_unlabeled(u, batch_size)
    s = s + del_s
    

Getting MNIST 3/8 classification data...
3/8 Training set size = 12001
3/8 Test set size = 1965
Initial Seed set size = 1200
Unlabelled dataset size = 10801
|S|_0 = 1200, |U|_0 = 10801
--------------------Start------------------
Training with seed size: 1200
Accuracy on the Test Set is : 0.1852
--------------------End------------------
|S|_1 = 2200, |U|_1 = 9801
--------------------Start------------------
Training with seed size: 2200
Accuracy on the Test Set is : 0.1861
--------------------End------------------
|S|_2 = 3200, |U|_2 = 8801
--------------------Start------------------
Training with seed size: 3200
Accuracy on the Test Set is : 0.1856
--------------------End------------------
|S|_3 = 4200, |U|_3 = 7801
--------------------Start------------------
Training with seed size: 4200
Accuracy on the Test Set is : 0.1846
--------------------End------------------
|S|_4 = 5200, |U|_4 = 6801
--------------------Start------------------
Training with seed size: 5200
Accuracy on the Test 